In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,472 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,016 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,250 kB]
Get:10 http://archive.ubuntu.com/ub

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# WebDriver 객체 생성
driver = webdriver.Chrome(options=options)

options.page_load_strategy = 'eager'


In [ ]:
from selenium.common.exceptions import TimeoutException

def crawl_factcheck_page(url, id):
    driver.get(url)

    try:
        # 원하는 요소가 로드될 때까지 대기
        WebDriverWait(driver, 5).until(
          EC.presence_of_element_located((By.CSS_SELECTOR, 'div.jsx-727853492.fact-dial-label-text'))
        )


    except TimeoutException:
        print("페이지 로딩 시간 초과 : ", id)
        return None
    except Exception as e:
        print(f"{id} - 오류 발생: {e}")
        return None

    # 페이지의 소스를 BeautifulSoup로 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    factdict = {}
    # 원하는 요소를 선택하여 크롤링
    factdict['id'] = id
    factdict['title'] = soup.find(class_='fact-lead-message').text
    factdict['source'] = soup.find(class_='fact-check-source').text[4:]
    factdict['judge'] = soup.find(class_='fact-dial-label-text').text
    try:
      factdict['date'] = soup.select_one('#__next > div > div.jsx-1740931948.layout-inner-container > div > div.jsx-727853492.page-inner-container > div.jsx-727853492.fact-show-score-card-continer > div.jsx-727853492.score-inner-container > div:nth-child(1) > select > option:nth-child(1)').text[5:].split()[1]
    except Exception as e:
      print(f"{id} - 오류 발생: {e}")
    resli = factdict
    #print(resli)
    return resli



In [ ]:
from tqdm.notebook import tqdm

id = 5194
factlist = []
for i in tqdm(range(id, 0, -1)):
  factlist.append(crawl_factcheck_page('https://factcheck.snu.ac.kr/facts/show?id=' + str(i), i))

In [ ]:
print(len(factlist))
print(factlist)


In [ ]:
import csv

print(factlist)

# CSV 파일 이름
csv_file = "/content/drive/MyDrive/itc2023_2_빅데이터/FactChecker/fact_checks.csv"

# CSV 파일 쓰기
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['id','date', 'title', 'source', 'judge'])
    writer.writeheader()  # CSV 파일의 헤더를 쓴다.
    for data in factlist:
      try:
        writer.writerow(data)  # 각 딕셔너리를 CSV 파일에 쓴다.
      except:
        pass
print(f"Data saved to {csv_file}")

In [ ]:
factlist